In [ ]:
!pip install recommenders

     |████████████████████████████████| 314 kB 5.4 MB/s 
     |████████████████████████████████| 636 kB 41.9 MB/s 
     |████████████████████████████████| 1.5 MB 38.6 MB/s 
     |████████████████████████████████| 2.9 MB 30.7 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 11.8 MB 53 kB/s 
     |████████████████████████████████| 61 kB 7.9 MB/s 
     |████████████████████████████████| 310 kB 53.5 MB/s 
     |████████████████████████████████| 12.4 MB 30 kB/s 
     |████████████████████████████████| 93 kB 2.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 39.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705374 sha256=e8ae90ebf6af987ec4d717a6c113e967d96861b6a2be4ee8298735a41bcc150a
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b1

In [ ]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 40.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=0bdcc748e736ef0b7406df79e29c6fce37bba47b5b1467d79a3103482c9bff75
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


### Global Setup

In [ ]:
import numpy as np
import pandas as pd
import recommenders

import sys
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Spark version: 3.1.2


Getting Data from drive. Not critical to overall project, but a helpful if running only on colab. Otherwise, local files are ok

In [ ]:
import gdown
book_tags_raw_url = 'https://drive.google.com/file/d/1zi-0f5-f9oOwgFGdoDAoTmcrkiPAqMaH/view?usp=sharing'
book_tags_url     ='https://drive.google.com/uc?id=' + book_tags_raw_url.split('/')[-2] #Do not change
gdown.download(book_tags_url, 'book_tags.csv',quiet=False)

books_raw_url = 'https://drive.google.com/file/d/1TcFgTOCoBBqfEwrSbhYqunu5qYDnzxSH/view?usp=sharing'
books_url     ='https://drive.google.com/uc?id=' + books_raw_url.split('/')[-2] #Do not change
gdown.download(books_url, 'books.csv',quiet=False)

ratings_raw_url = 'https://drive.google.com/file/d/12noQhcRgAYBUNcbhG2Z3fuNWKX0EebMA/view?usp=sharing'
ratings_url     ='https://drive.google.com/uc?id=' + ratings_raw_url.split('/')[-2] #Do not change
gdown.download(ratings_url, 'ratings.csv',quiet=False)

tags_raw_url = 'https://drive.google.com/file/d/1_4450oLYhCViIAQ2q2Y_SNcySufEETln/view?usp=sharing'
tags_url     ='https://drive.google.com/uc?id=' + tags_raw_url.split('/')[-2] #Do not change
gdown.download(tags_url, 'tags.csv',quiet=False)

to_read_raw_url = 'https://drive.google.com/file/d/1R-3wjdPjlpM5EH9TbDPt0Pm2AHQGZR34/view?usp=sharing'
to_read_url     ='https://drive.google.com/uc?id=' + to_read_raw_url.split('/')[-2] #Do not change
gdown.download(to_read_url, 'to_read.csv',quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1zi-0f5-f9oOwgFGdoDAoTmcrkiPAqMaH
To: /content/book_tags.csv
16.7MB [00:00, 88.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TcFgTOCoBBqfEwrSbhYqunu5qYDnzxSH
To: /content/books.csv
3.29MB [00:00, 134MB/s]
Downloading...
From: https://drive.google.com/uc?id=12noQhcRgAYBUNcbhG2Z3fuNWKX0EebMA
To: /content/ratings.csv
72.1MB [00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_4450oLYhCViIAQ2q2Y_SNcySufEETln
To: /content/tags.csv
100%|██████████| 722k/722k [00:00<00:00, 65.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1R-3wjdPjlpM5EH9TbDPt0Pm2AHQGZR34
To: /content/to_read.csv
9.41MB [00:00, 61.2MB/s]


'to_read.csv'

### Data Exploration

In [ ]:
df_ratings = pd.read_csv("ratings.csv")
print(df_ratings.shape)
df_ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [ ]:
df_books = pd.read_csv("books.csv")
print(df_books.shape)
df_books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
df_book_tags = pd.read_csv("book_tags.csv")
print(df_book_tags.shape)
df_book_tags.head()

(999912, 3)


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [ ]:
df_tags = pd.read_csv("tags.csv")
print(df_tags.shape)
df_tags.head()

(34252, 2)


,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [ ]:
df_to_read = pd.read_csv("to_read.csv")
print(df_to_read.shape)
df_to_read.head()

(912705, 2)


,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


### Set up Spark context

In [ ]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = start_or_get_spark("ALS PySpark", memory="16g")

In [ ]:
# top k items to recommend
TOP_K = 10

### Create the train and test dataset using spark

In [ ]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("user_id", IntegerType()),
        StructField("book_id", IntegerType()),
        StructField("rating", IntegerType()),
        # StructField("Timestamp", LongType()),
    )
)


In [ ]:
df_ratings_train, df_ratings_test = python_random_split(df_ratings, ratio=0.80, seed=42)

In [ ]:
train = spark.createDataFrame(df_ratings_train[['user_id', 'book_id', 'rating']], schema=schema)
train.show()

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|  10714|   7164|     3|
|  48091|   2213|     3|
|   9809|   5769|     4|
|  25191|     86|     5|
|  25441|   4884|     3|
|  29639|     20|     2|
|  28648|   2007|     5|
|  53402|   1122|     4|
|  42999|     45|     5|
|  44240|    373|     3|
|  38341|    862|     5|
|  27949|   7787|     5|
|  12813|     41|     2|
|  10629|   6845|     4|
|  33950|   3003|     5|
|  15872|    420|     4|
|  36752|    469|     3|
|  22840|   3994|     5|
|  24805|   9443|     4|
|   1752|   4663|     3|
+-------+-------+------+
only showing top 20 rows



In [ ]:
test = spark.createDataFrame(df_ratings_test[['user_id', 'book_id', 'rating']], schema=schema)
test.show()

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|  42562|   2757|     3|
|  43232|    134|     4|
|  37244|   1463|     5|
|  53366|     71|     2|
|  29634|   3339|     4|
|   2268|   2250|     4|
|  36766|     39|     4|
|  18377|   1544|     4|
|  41168|    182|     5|
|  14453|     35|     4|
|  42047|    227|     5|
|  42864|    482|     3|
|  27058|    737|     4|
|  33937|   6462|     5|
|   6400|   4427|     2|
|  48534|     78|     3|
|  39681|   2119|     5|
|  40068|    245|     3|
|  27520|   3966|     3|
|  41826|    361|     5|
+-------+-------+------+
only showing top 20 rows



### Train the ALS model on the training data, and get the top-k recommendations for our testing data

In [ ]:
header = {
    "userCol": "user_id",
    "itemCol": "book_id",
    "ratingCol": "rating",
}


als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

In [ ]:
with Timer() as train_time:
    model = als.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

Took 76.94046355800003 seconds for training.


In [ ]:
with Timer() as test_time:

    # Get the cross join of all user-item pairs and score them.
    users = train.select('user_id').distinct()
    items = train.select('book_id').distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)

    # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (dfs_pred['user_id'] == train['user_id']) & (dfs_pred['book_id'] == train['book_id']),
        how='outer'
    )

    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.rating"].isNull()) \
        .select('pred.' + 'user_id', 'pred.' + 'book_id', 'pred.' + "prediction")

    # In Spark, transformations are lazy evaluation
    # Use an action to force execute and measure the test time 
    top_all.cache().count()

print("Took {} seconds for prediction.".format(test_time.interval))

Took 3316.691716356 seconds for prediction.


In [ ]:
top_all.show()

+-------+-------+----------+
|user_id|book_id|prediction|
+-------+-------+----------+
|      1|    587| 3.0064232|
|      1|    869|  3.282434|
|      1|   1208| 2.9063702|
|      1|   1348| 4.1920395|
|      1|   1357| 3.2595832|
|      1|   1677|  2.746942|
|      1|   1702|  2.461098|
|      1|   1720| 3.3358562|
|      1|   1892|  3.306596|
|      1|   2086|  4.051029|
|      1|   2202| 2.4670594|
|      1|   2324| 3.4170425|
|      1|   2483|  1.552057|
|      1|   2545| 2.5770388|
|      1|   2667|  3.597023|
|      1|   2870|  2.928708|
|      1|   3304|  3.285502|
|      1|   3452| 2.3967571|
|      1|   3468|  2.905528|
|      1|   3477| 2.1990151|
+-------+-------+----------+
only showing top 20 rows



### Evaluate how well ALS performs

In [ ]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user="user_id", col_item="book_id", 
                                    col_rating="rating", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.000432
NDCG:	0.003319
Precision@K:	0.003719
Recall@K:	0.001734


### Evaluate rating prediction

In [ ]:
# Generate predicted ratings.
prediction = model.transform(test)
prediction.cache().show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  10206|    148|     5|  3.862525|
|  11317|    148|     4| 3.4474127|
|  17679|    148|     4| 3.7459688|
|  24354|    148|     5| 4.1379323|
|  28170|    148|     4| 3.3091235|
|  33569|    148|     5| 3.8182042|
|  29827|    148|     4| 4.3291073|
|  37734|    148|     4| 4.4295387|
|  38707|    148|     4| 3.6138372|
|  47268|    148|     4| 3.2125015|
|  51801|    148|     4| 3.9465508|
|   8257|    148|     4| 3.0405352|
|  14377|    148|     3| 3.1132224|
|  17837|    148|     3| 3.6867065|
|  20467|    148|     3| 3.1939578|
|  25686|    148|     5|  4.160081|
|  38146|    148|     4| 2.8748088|
|  52987|    148|     4| 3.9128532|
|  27856|    148|     4| 3.9172032|
|  29225|    148|     4| 3.8288894|
+-------+-------+------+----------+
only showing top 20 rows



In [ ]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user="user_id", col_item="book_id", 
                                    col_rating="rating", col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	0.816477
MAE:	0.638406
Explained variance:	0.329098
R squared:	0.320281


### Show predictions in pandas

In [ ]:
# prediction for user 1
resultUser1 = top_all[top_all['User_ID'] == 1]

In [ ]:
resultUser1 = resultUser0.sort(resultUser0.prediction.desc())

In [ ]:
resultUser1.show()

+-------+-------+----------+
|user_id|book_id|prediction|
+-------+-------+----------+
|      1|   8946| 5.0081735|
|      1|   8014|  4.941686|
|      1|   9076| 4.9302373|
|      1|   7537| 4.9099874|
|      1|   3628| 4.8529167|
|      1|   7254| 4.8218365|
|      1|   4868| 4.8018284|
|      1|   5580| 4.7939496|
|      1|   9566| 4.7528443|
|      1|   7593|  4.715734|
|      1|   4483|  4.713971|
|      1|   9392| 4.7129197|
|      1|   6920| 4.7020187|
|      1|   5207|  4.701923|
|      1|   8548|  4.682682|
|      1|   9531| 4.6770654|
|      1|   6590|  4.666998|
|      1|   9569|  4.655827|
|      1|   1198| 4.6499124|
|      1|   6361|  4.634282|
+-------+-------+----------+
only showing top 20 rows



In [ ]:
df_resultUser1 = resultUser1.select("*").toPandas()

In [ ]:
df_resultUser1.head(n=10)

,user_id,book_id,prediction
0,1,8946,5.008173
1,1,8014,4.941686
2,1,9076,4.930237
3,1,7537,4.909987
4,1,3628,4.852917
5,1,7254,4.821836
6,1,4868,4.801828
7,1,5580,4.793950
8,1,9566,4.752844
9,1,7593,4.715734


In [ ]:
# merge top 10 prediction with activities dataframe
df_resultUser1 = pd.merge(df_resultUser1, df_books, on='book_id', how='left')

In [ ]:
print("User 1 top 10 predictions")
df_resultUser1[['prediction', 'user_id', 'book_id', 'title', 'isbn']].head(n=10)

User 1 top 10 predictions


,prediction,user_id,book_id,title,isbn
0,5.008173,1,8946,The Divan,9646534783
1,4.941686,1,8014,"Psycho-Cybernetics, A New Way to Get More Livi...",671700758
2,4.930237,1,9076,Preach My Gospel: A Guide To Missionary Service,402366174
3,4.909987,1,7537,Citizen: An American Lyric,1555976905
4,4.852917,1,3628,The Complete Calvin and Hobbes,740748475
5,4.821836,1,7254,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,836217691
6,4.801828,1,4868,Jesus the Christ,875793266
7,4.793950,1,5580,The Calvin and Hobbes Lazy Sunday Book,836218523
8,4.752844,1,9566,Attack of the Deranged Mutant Killer Monster S...,1417642165
9,4.715734,1,7593,"This Is Water: Some Thoughts, Delivered on a S...",316068225
